In [2]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [3]:
from urllib.parse import quote
import latextools
import drawsvg as draw
def convertFormulaToSVGURL(formula):
    latex_eq = latextools.render_snippet(
        "$" + formula + "$",
        commands=[latextools.cmd.all_math])
    svg_eq = latex_eq.as_svg()
    svgwidth = 300 + 5 * svg_eq.width
    svgheight = 100 + 5 * svg_eq.height
    d = draw.Drawing(svgwidth, svgheight, origin='center', displayInline=False)
    d.append(draw.Ellipse(0, 0, svgwidth/2 - 1, svgheight/2 - 1, fill='yellow', stroke='black', stroke_width=2))
    scale = 0.95/ ((svg_eq.width / svgwidth) ** 2 + (svg_eq.height / svgheight) ** 2) ** 0.5
    d.draw(svg_eq, x=0, y=0, center=True, scale=scale)
    simpleSvg = d.as_svg()
    return f"data:image/svg+xml,{quote(simpleSvg)}"

In [4]:
def convertFormulaSymbolDefinitionToSVGURL(symbol):
    latex_eq = latextools.render_snippet(
        "where $" + symbol + "$ is",
        commands=[latextools.cmd.all_math])
    svg_eq = latex_eq.as_svg()
    # Fit the symbol definition into a rectangle
    d = draw.Drawing(svg_eq.width + 6, svg_eq.height + 6, origin='center', displayInline=False)
    d.append(draw.Rectangle(-d.width/2, -d.height/2, d.width, d.height, fill='lightgreen', stroke='black', stroke_width=2))
    d.draw(svg_eq, x=0, y=0, center=True)
    simpleSvg = d.as_svg()
    return f"data:image/svg+xml,{quote(simpleSvg)}"

In [5]:
class VisNetwork:
    def __init__(self):
        self.nodes = []
        self.edges = []
        self.idCounter = 0
    def addNode(self, style = {}, label = None):
        node = VisNode(self, self.idCounter, style, label)
        self.idCounter += 1
        self.nodes.append(node)
        return node
    def addEquationNode(self, equation, isEllipse = True):
        node = VisNode(self, self.idCounter, {}, "", equation, "ellipse" if isEllipse else "rectangle")
        self.idCounter += 1
        self.nodes.append(node)
        return node
    def addEdge(self, fromNode, toNode, style = {}, label = None):
        edge = VisEdge(self, fromNode, toNode, style, label)
        self.idCounter += 1
        self.edges.append(edge)
        return edge
    def getVisNetworkJSON(self):
        nodes = [node.getVisNodeJSON() for node in self.nodes]
        edges = [edge.getVisEdgeJSON() for edge in self.edges]
        return {"nodes": nodes, "edges": edges}
    def saveToHTMLFile(self, htmlFilePath):
        safeVisNetworkJSONToHTMLFile(self.getVisNetworkJSON(), htmlFilePath)
        

class VisNode:
    def __init__(self, network, id, style = {}, label = None, equation = None, equationStyle = "ellipse"):
        self.id = id
        self.label = label
        self.network = network
        self.style = style
        assert not type(style) is str
        self.equation = equation
        self.equationStyle = equationStyle
    def getVisNodeJSON(self):
        jsonValue = {} | self.style | {"id": self.id}
        if self.label:
            jsonValue |= {"label": self.label}
        if self.equation:
            equationURL = convertFormulaToSVGURL(self.equation) if self.equationStyle == "ellipse" else convertFormulaSymbolDefinitionToSVGURL(self.equation)
            jsonValue |= {"shape": "image", "image": equationURL}
            if self.equationStyle == "rectangle":
                jsonValue |= {"size": 11}
        return jsonValue

class VisEdge:
    def __init__(self, network, fromNode, toNode, style = {}, label = None):
        self.fromNode = fromNode
        self.toNode = toNode
        self.label = label
        self.network = network
        self.style = style
    def getVisEdgeJSON(self):
        jsonValue = {} | self.style | {"from": self.fromNode.id, "to": self.toNode.id}
        if self.label:
            jsonValue |= {"label": self.label}
        return jsonValue

In [24]:
def addEquation(network, leftSide, rightSide):
    leftNode = leftSide if type(leftSide) is VisNode else network.addNode({"shape": "image", "image": convertFormulaToSVGURL(leftSide)})
    rightNode = rightSide if type(rightSide) is VisNode else network.addNode({"shape": "image", "image": convertFormulaToSVGURL(rightSide)})
    equalNode = network.addNode({"font": {"size": 16}, "color" : {"background": "rgba(255, 100, 100, 255)", "border": "darkred"},"shape" : "box"}, "equals")
    network.addEdge( equalNode, leftNode,{"color": "rgba(200, 60, 60, 255)", "width": 5, "arrows": "to"})
    network.addEdge(equalNode, rightNode, {"color": "rgba(200, 60, 60, 255)", "width": 5, "arrows": "to"})
    return (leftNode, equalNode, rightNode)
conceptStyle = {"font": {"size": 20}, 
                "color" : {"background": "rgba(140, 220, 255, 255)", 
                "border": "rgba(0, 0, 255, 255)"}}
def addSymbolDefinition(network, term, symbol, definition):
    termNode = term if type(term) is VisNode else network.addNode({"shape": "image", "image": convertFormulaToSVGURL(term)})
    symbolNode = symbol if type(symbol) is VisNode else network.addNode({"shape": "image", "image": convertFormulaSymbolDefinitionToSVGURL(symbol), "size": 11})
    definitionNode = definition if type(definition) is VisNode else network.addNode(conceptStyle, definition)
    network.addEdge(termNode, symbolNode, {"color": "rgba(100, 200, 60, 255)", "width": 3})
    network.addEdge(symbolNode, definitionNode, {"color": "rgba(100, 200, 60, 255)", "width": 3, "arrows": "to"})
    return (termNode, symbolNode, definitionNode)
def addFromThisFollowsThat(network, thisArray, thatArray):
    thisNodes = [node if type(node) is VisNode else network.addNode({"shape": "image", "image": convertFormulaToSVGURL(node)}) for node in thisArray]
    thatNodes = [node if type(node) is VisNode else network.addNode({"shape": "image", "image": convertFormulaToSVGURL(node)}) for node in thatArray]
    followNode = network.addNode({"color": {"background": "rgba(255, 140, 220, 255)"}}, "from this\nfollows that")
    for thisNode in thisNodes:
        network.addEdge(thisNode, followNode, {"color": "rgba(255, 140, 220, 255)", "width": 3, "arrows": "to"})
    for thatNode in thatNodes:
        network.addEdge(followNode, thatNode, {"color": "rgba(255, 140, 220, 255)", "width": 3, "arrows": "to"})
    return (thisNodes, thatNodes)
def addEquationDescription(network, equation, description):
    equationNode = equation
    descriptionNode = description if type(description) is VisNode else network.addNode(conceptStyle, description)
    network.addEdge(equationNode, descriptionNode, {"color": "rgba(100, 200, 60, 255)", "width": 3, "arrows": "to"})
    return (equationNode, descriptionNode)
def addSentence(network, sentenceArray):
    sentenceHead = network.addNode({"shape": "diamond", "size": 20, "color": {"background": "rgb(255, 70, 70)"}})
    lastNode = sentenceHead
    returnArray = [sentenceHead]
    for sentenceComponent in sentenceArray:
        if type(sentenceComponent) is list and len(sentenceComponent) == 1 and type(sentenceComponent[0]) is str:
            connectingNode = network.addNode({"shape": "box", "color": {"background": "rgb(255, 230, 34)"}, "font": {"size": 15}}, sentenceComponent[0])
        else:
            connectingNode = network.addNode({"shape": "dot", "size": 10, "color": {"background": "rgb(255, 130, 70)"}})
            componentNode = sentenceComponent if type(sentenceComponent) is VisNode else network.addNode(conceptStyle, sentenceComponent)
            network.addEdge(connectingNode, componentNode, {"arrows": "from", "color": {"color": "gray"}, "width" : 3})
            returnArray.append(componentNode)
        network.addEdge(lastNode, connectingNode, { "width": 10, "arrows": "to", "color": "rgb(255, 180, 70)"})
        lastNode = connectingNode
    return returnArray
def addIsSpecialCaseOf(network, specialCase, generalCase):
    generalNode = generalCase if type(generalCase) is VisNode else network.addNode(conceptStyle, generalCase)
    specialNode = specialCase if type(specialCase) is VisNode else network.addNode(conceptStyle, specialCase)
    isSpecialCaseOfNode = network.addNode({"shape": "box", "color": {"background": "rgb(116, 220, 180)"}}, "is special\ncase of")
    network.addEdge(isSpecialCaseOfNode, generalNode, {"color": "rgb(90, 250, 190)", "width": 3, "arrows": "to"})
    network.addEdge(isSpecialCaseOfNode, specialNode, {"color": "rgb(90, 250, 190)", "width": 3})
    return (generalNode, isSpecialCaseOfNode, specialNode)
def addIsListOf(network, specialCase, generalCase):
    generalNode = generalCase if type(generalCase) is VisNode else network.addNode(conceptStyle, generalCase)
    specialNode = specialCase if type(specialCase) is VisNode else network.addNode(conceptStyle, specialCase)
    isSpecialCaseOfNode = network.addNode({"shape": "box", "color": {"background": "rgb(116, 220, 180)"}}, "is list of")
    network.addEdge(isSpecialCaseOfNode, generalNode, {"color": "rgb(90, 250, 190)", "width": 3, "arrows": "to"})
    network.addEdge(isSpecialCaseOfNode, specialNode, {"color": "rgb(90, 250, 190)", "width": 3})
    return (generalNode, isSpecialCaseOfNode, specialNode)
def addConceptNode(network, concept):
    return network.addNode(conceptStyle, concept)
def addTriple(network, subject, predicate, object):
    subjectNode = subject if type(subject) is VisNode else network.addNode(conceptStyle, subject)
    predicateNode = predicate if type(predicate) is VisNode else network.addNode({"shape": "box", "color": {"background": "rgb(255, 200, 100)"}}, predicate)
    objectNode = object if type(object) is VisNode else network.addNode(conceptStyle, object)
    network.addEdge(subjectNode, predicateNode, {"color": "rgb(160, 190, 250)", "width": 3, "arrows": "to"})
    network.addEdge(predicateNode, objectNode, {"color": "rgb(160, 190, 250)", "width": 3, "arrows": "to"})
    return (subjectNode, predicateNode, objectNode)

In [37]:
nw = VisNetwork()
showSymbolDefinition = False
ham = False
leftSchroed, schroedEq, rightSchroed = addEquation(nw, "i\\hbar\\frac{\\partial}{\\partial t}\\psi(x,t)", "-\\frac{\\hbar^2}{2m}\\Delta\\psi(x,t)+V(x)\\psi(x,t)")
if showSymbolDefinition:
    _, _, waveFunc = addSymbolDefinition(nw, leftSchroed, "\\psi(x,t)", "wave function\n of a particle in a \ntime-independent potential")
    addSymbolDefinition(nw, rightSchroed, "\\psi(x,t)", waveFunc)
    _, _, time = addSymbolDefinition(nw, leftSchroed, "t", "time")
    addSymbolDefinition(nw, rightSchroed, "t", time)
    _, _, x = addSymbolDefinition(nw, leftSchroed, "x", "position of the particle")
    addSymbolDefinition(nw, rightSchroed, "x", x)
    _, _, hbar = addSymbolDefinition(nw, leftSchroed, "\\hbar", "reduced Planck constant")
    addSymbolDefinition(nw, rightSchroed, "\\hbar", hbar)
    _, _, mass = addSymbolDefinition(nw, rightSchroed, "m", "mass of the particle")
    _, _, vPotential = addSymbolDefinition(nw, rightSchroed, "V(x)", "potential energy of the particle")
    addEquationDescription(nw, schroedEq, "Schroedinger equation")

leftSeperation, seperationEq, rightSeperation = addEquation(nw, "\\psi(x,t)", "\\varphi(x)\\chi(t)")
if showSymbolDefinition:
    _, _, statWaveFunc = addSymbolDefinition(nw, leftSeperation, "\\psi(x,t)", "stationary wave function\n of a particle in a \ntime-independent potential")
    addIsSpecialCaseOf(nw, statWaveFunc, waveFunc)
    _, _, phi = addSymbolDefinition(nw, rightSeperation, "\\varphi(x)", "spatial part of the wave function")
    _, _, chi = addSymbolDefinition(nw, rightSeperation, "\\chi(t)", "temporal part of the wave function")

leftTimeDepSchroed, timeDepSchroed, varSepConst= addEquation(nw, "\\frac{i\\hbar}{\\chi(t)}\\frac{d\\chi(t)}{dt}", "E")
if showSymbolDefinition:
    pass
leftSpatialSchroed, spatialSchroed, _ = addEquation(nw, "-\\frac{\\hbar^2}{2m \\varphi(x)}\\Delta\\varphi(x) + V(x)", varSepConst)
addFromThisFollowsThat(nw, [schroedEq, seperationEq], [timeDepSchroed, spatialSchroed])

leftEnergyAssertion, energyAssertion, _ = addEquation(nw, "\\hbar\\omega", varSepConst)

leftSolutionOfChi, solutionOfChi, rightSolutionOfChi = addEquation(nw, "\\chi(t)", "Ae^{-i\\omega t}")
if showSymbolDefinition:
    _, _, omega = addSymbolDefinition(nw, rightSolutionOfChi, "\\omega", "angular frequency")
    _, _, A = addSymbolDefinition(nw, rightSolutionOfChi, "A", "complex constant")
addFromThisFollowsThat(nw, [timeDepSchroed, energyAssertion], [solutionOfChi])

if ham:
    leftDiffEqOfPhi, diffEqOfPhi, rightDiffEqOfPhi = addEquation(nw, "-\\frac{\\hbar^2}{2m}\\Delta\\varphi(x) + V\\varphi(x)", "\\hbar\\omega\\varphi(x)")
    addFromThisFollowsThat(nw, [spatialSchroed, energyAssertion], [diffEqOfPhi])

    leftHamDef, hamDef, rightHamDef = addEquation(nw, "H", "-\\frac{\\hbar^2}{2m}\\Delta + V(x)")
    _, hamDiff, rightHamDif = addEquation(nw, leftDiffEqOfPhi, "H\\varphi(x)")
    addFromThisFollowsThat(nw, [hamDef], [hamDiff])

_, statSol, rightStatSol = addEquation(nw, leftSeperation, "\\varphi(x)e^{-i\\omega t}")
addFromThisFollowsThat(nw, [seperationEq, solutionOfChi], [statSol])

leftLinComb, linComb, rightLinComb = addEquation(nw, "\\psi(x,t)", "\\sum_n \\psi_n(x, t)")
addFromThisFollowsThat(nw, [schroedEq], [linComb])

_, solution, rightSolution = addEquation(nw, leftLinComb, "\\sum_n c_n\\varphi_n(x)e^{-i\\omega_n t}")
addFromThisFollowsThat(nw, [linComb, statSol], [solution])

nw.saveToHTMLFile("output.html")

In [38]:
nw = VisNetwork()

forceTerm, newtonEq, massAccelTerm = addEquation(nw, "F", "m \\frac{d^2x}{dt^2}")
forceTerm, gravEq, gravForceTerm = addEquation(nw, "F", "g m")
gravTerm, gravForceEq, gravAccelTerm = addEquation(nw, "g", "\\frac{d^2x}{dt^2}")
addFromThisFollowsThat(nw, [newtonEq, gravEq], [gravForceEq])
xTerm, solvedDifEq, xFunc = addEquation(nw, "x(t)", "x_0 + v_0 t + \\frac{1}{2} g t^2")
addFromThisFollowsThat(nw, [gravForceEq], [solvedDifEq])

nw.saveToHTMLFile("output.html")


In [39]:
nw = VisNetwork()

forceTerm1, newtonEq, massAccelTerm = addEquation(nw, "F", "m \\frac{d^2x}{dt^2}")
_, _, forceExpl1 = addSymbolDefinition(nw, forceTerm1, "F", "force acting\non a mass")
forceTerm2, gravEq, gravForceTerm = addEquation(nw, "F", "g m")
_, _, forceExpl2 = addSymbolDefinition(nw, forceTerm2, "F", "gravitational\nforce acting\non a mass")
forceTerm3, springEq, springForceTerm = addEquation(nw, "F", "-k x")
_, _, forceExpl3 = addSymbolDefinition(nw, forceTerm3, "F", "spring force\nacting on a mass")
addIsSpecialCaseOf(nw, forceExpl2, forceExpl1)
addIsSpecialCaseOf(nw, forceExpl3, forceExpl1)
addFromThisFollowsThat(nw, [newtonEq, gravEq], [addConceptNode(nw, ". . .")])
addFromThisFollowsThat(nw, [newtonEq, springEq], [addConceptNode(nw, ". . .")])

nw.saveToHTMLFile("output.html")

In [40]:
nw = VisNetwork()

wave = addConceptNode(nw, "wave")
boundaryBeamDiameter, beamDiameter, wave, boundary = addSentence(nw, ["beam diameter", ["of the"], wave, ["at the"], "boundary"])
addSentence(nw, [wave, ["is passing through the"], boundary])

incomingWave, wave = addSentence(nw, [["incoming"], wave])
incomingWavefront, wavefront, incomingWave = addSentence(nw, ["front", ["of the"], incomingWave])
incomingTriangle, triangle, incomingWave = addSentence(nw, [["right"], "triangle", ["of the"], incomingWave])
incomingWavefrontCathetus, incomingWavefront, incomingTriangle = addSentence(nw, [incomingWavefront, ["is cathetus of"], incomingTriangle])
incomingTravelDistance, travelDistance, incomingWave, timeIntervall = addSentence(nw, ["travel distance", ["of the"], incomingWave, ["during"], "time interval delta t"])
incomingTravelDiatanceCathetus, incomingTravelDistance, incomingTriangle = addSentence(nw, [incomingTravelDistance, ["is cathetus of"], incomingTriangle])
incomingBeamDiameterHypotenuse, boundaryBeamDiameter, incomingTriangle = addSentence(nw, [boundaryBeamDiameter, ["is hypotenuse of"], incomingTriangle])
incomingNormalVector, normalVector, boundary, incomingWave = addSentence(nw, ["normal vector", ["of the"], boundary, ["in direction of the"], incomingWave])
incomingAngle, angle, incomingWavefront, normalVector = addSentence(nw, ["angle", ["between"], incomingTravelDistance, ["and"], incomingNormalVector])
incomingBoundaryAngle, angle, incomingWavefront, boundaryBeamDiameter = addSentence(nw, [angle, ["between"], incomingTravelDistance, ["and"], boundaryBeamDiameter])
addSentence(nw, [incomingBoundaryAngle, ["is equal to"], incomingAngle])
velocityOfIncomingWave, velocity, incomingWave = addSentence(nw, ["velocity", ["of the"], incomingWave])

outgoingWave, wave = addSentence(nw, [["outgoing"], "wave"])
outgoingWavefront, wavefront, outgoingWave = addSentence(nw, [wavefront, ["of the"], outgoingWave])
outgoingTriangle, triangle, outgoingWave = addSentence(nw, [["right"], triangle, ["of the"], outgoingWave])
outgoingWavefrontCathetus, outgoingWavefront, outgoingTriangle = addSentence(nw, [outgoingWavefront, ["is cathetus of"], outgoingTriangle])
outgoingTravelDistance, travelDistance, outgoingWave, timeIntervall = addSentence(nw, [travelDistance, ["of the"], outgoingWave, ["during"], timeIntervall])
outgoingTravelDiatanceCathetus, outgoingTravelDistance, outgoingTriangle = addSentence(nw, [outgoingTravelDistance, ["is cathetus of"], outgoingTriangle])
outgoingBeamDiameterHypotenuse, boundaryBeamDiameter, outgoingTriangle = addSentence(nw, [boundaryBeamDiameter, ["is hypotenuse of"], outgoingTriangle])
outgoingNormalVector, normalVector, boundary, outgoingWave = addSentence(nw, [normalVector, ["of the"], boundary, ["in direction of the"], outgoingWave])
outgoingAngle, angle, outgoingWavefront, normalVector = addSentence(nw, [angle, ["between"], outgoingTravelDistance, ["and"], outgoingNormalVector])
outgoingBoundaryAngle, angle, outgoingWavefront, boundaryBeamDiameter = addSentence(nw, [angle, ["between"], outgoingTravelDistance, ["and"], boundaryBeamDiameter])
addSentence(nw, [outgoingBoundaryAngle, ["is equal to"], outgoingAngle])
velocityOfOutgoingWave, velocity, outgoingWave = addSentence(nw, [velocity, ["of the"], outgoingWave])

d1, travelDistanceEquation1, v1DeltaT = addEquation(nw, "d_1", "v_1 \\Delta t")
addSymbolDefinition(nw, d1, "d_1", incomingTravelDistance)
addSymbolDefinition(nw, v1DeltaT, "v_1", velocityOfIncomingWave)
addSymbolDefinition(nw, v1DeltaT, "\\Delta t", timeIntervall)
sintheta1, sinEquation1, fracV1DeltaTB = addEquation(nw, "sin\\Theta_1", "\\frac{v_1 \\Delta t}{b}")
addSymbolDefinition(nw, sintheta1, "\\Theta_1", incomingBoundaryAngle)
addSymbolDefinition(nw, fracV1DeltaTB, "v_1", velocityOfIncomingWave)
addSymbolDefinition(nw, fracV1DeltaTB, "\\Delta t", timeIntervall)
addSymbolDefinition(nw, fracV1DeltaTB, "b", boundaryBeamDiameter)
addFromThisFollowsThat(nw, [travelDistanceEquation1, incomingWavefrontCathetus, incomingBeamDiameterHypotenuse], [sinEquation1])

d2, travelDistanceEquation2, v2DeltaT = addEquation(nw, "d_2", "v_2 \\Delta t")
addSymbolDefinition(nw, d2, "d_2", outgoingTravelDistance)
addSymbolDefinition(nw, v2DeltaT, "v_2", velocityOfOutgoingWave)
addSymbolDefinition(nw, v2DeltaT, "\\Delta t", timeIntervall)
sintheta2, sinEquation2, fracV2DeltaTB = addEquation(nw, "sin\\Theta_2", "\\frac{v_2 \\Delta t}{b}")
addSymbolDefinition(nw, sintheta2, "\\Theta_2", outgoingBoundaryAngle)
addSymbolDefinition(nw, fracV2DeltaTB, "v_2", velocityOfOutgoingWave)
addSymbolDefinition(nw, fracV2DeltaTB, "\\Delta t", timeIntervall)
addSymbolDefinition(nw, fracV2DeltaTB, "b", boundaryBeamDiameter)
addFromThisFollowsThat(nw, [travelDistanceEquation2, outgoingWavefrontCathetus, outgoingBeamDiameterHypotenuse], [sinEquation2])

fracTheta1Theta2, snellsLaw, fracV1V2 = addEquation(nw, "\\frac{\\Theta_1}{\\Theta_2}", "\\frac{v_1}{v_2}")
addSymbolDefinition(nw, fracTheta1Theta2, "\\Theta_1", incomingBoundaryAngle)
addSymbolDefinition(nw, fracTheta1Theta2, "\\Theta_2", outgoingBoundaryAngle)
addSymbolDefinition(nw, fracV1V2, "v_1", velocityOfIncomingWave)
addSymbolDefinition(nw, fracV1V2, "v_2", velocityOfOutgoingWave)
addFromThisFollowsThat(nw, [sinEquation1, sinEquation2], [snellsLaw])

nw.saveToHTMLFile("output.html")